# Project 2: Wrangle OpenStreetMap 

## Author: Félix Carlos Camacho Criado

## Table of Contents
<ul>
<li><a href="#sum">Summary</a></li>
<li><a href="#map">Map Area</a></li>
<li><a href="#eda">Audit phase</a></li>
<li><a href="#dc">Data Cleaning phase</a></li>
<li><a href="#over">Data Overview</a></li>
<li><a href="#add">Additional ideas about the dataset</a></li>
<li><a href="#conc">Conclusion</a></li>  
</ul>

<a id='sum'></a>
## Summary

>In this project I will implement the techniques learnt in the Data Wrangling with SQL section. First, I will use the Overpass API to download a XML OSM dataset from https://www.openstreetmap.org. Then, I will proccess the dataset and audit it and remark the most noticeable findings and problems. After this, I will show some alternatives in order to
fix these issues and update data in an appropiate way.

>Once I have a clean dataset I will convert it from XML to CSV format, so, the cleaned .csv files can be import into a SQL database using the schema provided. After building my local database, there will be some time for data exploration and present additional ideas for improving the data quality of the dataset. In the last paragraph, you can find the conclusion that I draw after the project execution.


<a id='map'></a>
## Map Area

### Location: Madrid, Metropolitan Area

>I have choosen the metropolitan area of Madrid, because include info from my home and also from my workplace. This dataset not only include of Madrid city but info about all the cities which compose its metropolitan area as Getafe (the city where I live) so I am curious about which are the data included for this area and what can be improved.

>Details about the location can be found in the following link: https://www.openstreetmap.org/relation/5326784

>This file (OSM_XLM_Madrid_City_Map.osm) have of size of 867MB. Definetely it is not the best option for code testing, so I also use a sample of this place which I named sample_Madrid.osm of 3,2 MB and which is a more appropiate and agile option.

 


<a id='eda'></a>
## Audit Phase

> During this phase, I will apply the audit techniques used in the case of study as this make me know about what kind of tags are in the dataset and which are the most common tags values. In a deeper audit of this dataset, I will remark the main issues identified and how they affect to the tags.


### Elements Type

First, I will take a look to the unique elements that can be found in the whole dataset:

In [1]:
%run 1_eCounter.py

[('nd', 4896861),
 ('node', 3565274),
 ('tag', 2255104),
 ('way', 571743),
 ('member', 212575),
 ('relation', 13611),
 ('osm', 1),
 ('note', 1),
 ('meta', 1),
 ('bounds', 1)]


### Tags values

Before processing the data and adding into the database, I will check "k" values and "v" value for each element and see if
there are any potential problems.

Following the exercise provided in the case of study, I will check for certain patterns in the tags, so I have distribute
all tags in four groups of regular expressions in order to know if the problematic characters are a significant number
, which is not happening.

In [2]:
%run 2a_TagsK.py

{'lower': 1541385, 'lower_colon': 679739, 'other': 33977, 'problemchars': 3}


Regarding the "v" values for each tag, these are most common values for tags in Madrid. Due to there a lot of kind of tags I am showing only the ones repeated more than 50k times.

In [3]:
%run 2b_TagsV.py

[('highway', 258195),
 ('building', 224060),
 ('name', 136735),
 ('addr:street', 90301),
 ('source', 89486),
 ('addr:housenumber', 88443),
 ('building:levels', 87952),
 ('addr:postcode', 71471),
 ('oneway', 58810),
 ('addr:city', 56975),
 ('natural', 53103),
 ('source:date', 52077),
 ('barrier', 51352)]


### Problems encountered in the map
I put the focus in the following key features: k = addr:street and k = addr:city. It seems that the users who contributed to include information of this area have used a different nomenclature for the kind of street and also for the names of cities and villages, therefore, I found several error that can be fixed.

#### Auditing Street Names 
The main problems encountered for this key are:
<ul>
<li><b>LowerCase and Uppercase</b></li> E.g "CARRETERA" or "carretera" Instead of "Carretera".     
<li><b>Abbreviations</b></li> E.g. "CR","Ctra","CTRA." Instead of "Carretera".   
<li><b>Misspelling</b></li> E.g. "Pasage" Instead of "Pasaje".   
<li><b>Unwanted characters</b></li> E.g. ",".   
<li><b>Several ways to name the same concept </b></li> E.g. I found 11 strings for the Spanish word for "Avenue" (Avenida) such as "A","AV.","AVDA.","AVDA","Avda.","Avd." etc..
</ul>

I have identify the issues related with street names running the file "3b_AuditStreetNames.py".

#### Auditing City Names 
I found some similarities in the issues between this key (addr:city) and the previous one (addr:street) but I also found new problems to deal with.
<ul>
<li><b>LowerCase and Uppercase</b></li> E.g "alcobendas" Instead of "Alcobendas"."MADRID" Instead of "Madrid".  
<li><b>Misspelling </b></li> E.g. "Madrd" Instead of "Madrid".
<li><b>Unwanted characters</b></li> E.g. ",".   
<li><b>Uncomplete/Incorrect names for determined cities and villages </b></li> E.g. "Rivas Vaciamadrid3" Instead of 
"Rivas-Vaciamadrid"
<li><b>Several way to name the same city.</b></li> E.g.
"Fuente el Saz de Jarama","Fuente el Saz del Jarama","FUENTE EL SAZ DE JARAMA" Instead of  "Fuente El Saz De Jarama".    
So, I tried to unified these names in one similar value.   

</ul>

I have identify the issues related with city names running the file "3a_AudityCityNames.py".


<a id='dc'></a>
## Data Cleaning phase

> Once some issues have been identified, I cleaned data in order to correct these issues before convert and split the OSM file in CSV files. So I define a function for the update that will be called when the key was equal to 'addr:street' or 'addr:city'. 

In [4]:
def update_name(name, mapping):
    
    unwanted = [',']  # List of unwanted characters 
    el = ''                  
 
    #remove unwanted characters
    for i in range(len(name)):
        if name[i] not in unwanted:
            el = el + name[i]

    #Capitalize the first letter of each element and put to lower case the rest of letters
    low_name = el.lower()
    if ' ' in low_name:
        el = ''
        l = low_name.split(' ')
        for i in l:
            el = el + ' ' + i.capitalize()
    else:
        el = low_name.capitalize()

    #Match with mapping dict and in case it found some know issue/value, it replaces it for the correct form.
    k = mapping.keys()
    key_list = list(k)
    for abrev in key_list:
        if abrev in el.split():
            el = el.replace(abrev,mapping[abrev])

    return el

>The whole process can be found in the file 4_CleanUpdate.py. In addition, this process also launch the creation of five .CSV with info from nodes, ways and tags and their relations between them. Let's take a look in one .CSV file.



In [5]:
import pandas as pd

nodes_tags = pd.read_csv("nodes_tags.csv") 
nodes_tags.head() #the header and values are which I expected.

,id,key,value,type
0,171946,crossing,traffic_signals,regular
1,171946,crossing_ref,zebra,regular
2,171946,highway,traffic_signals,regular
3,20952908,crossing,traffic_signals,regular
4,20952908,highway,crossing,regular


>Now, .CSV files include cleaned data, next step is to create the db structure where the files will be imported. This step can be found in the file "5_CreateDB.py". Once the info is uploaded I will play some queries and try to obtain some basic statistics.

<a id='over'></a>
## Data Overview

>This section includes a information about the size of the .CSV files which has been uploaded to the file "Madrid.db". I will have a look into the file sizes, the number of nodes and ways, as well as, information about the users who contributed to fill the data about the metropolitan area of Madrid. 


In [6]:
import os
cwd = os.getcwd()
for root, dirs, files in os.walk(cwd+'/files_p2', topdown=False):
    for name in files:
        f = os.path.join(root, name)
        print (name.ljust(20), round(os.path.getsize(f)/1000000, 2), 'MB')

ways_tags.csv        49.08 MB
nodes_tags.csv       30.33 MB
nodes.csv            299.47 MB
Madrid.db            501.36 MB
sample_Madrid.osm    3.24 MB
OSM_XLM_Madrid_City_Map.osm 866.94 MB
ways.csv             35.09 MB
ways_nodes.csv       117.64 MB


### Number of nodes

In [7]:
import sqlite3

db = sqlite3.connect("Madrid.db")
c = db.cursor()

# Fetch records from either Madrid.db
QUERY = "SELECT COUNT(*) FROM nodes;"
c.execute(QUERY)

rows = c.fetchall()

print("Number of nodes:",rows)

Number of nodes: [(3565273,)]


### Number of ways

In [8]:
QUERY = "SELECT COUNT(*) FROM ways;"
c.execute(QUERY)

rows = c.fetchall()

print("Number of ways:",rows)

Number of ways: [(571741,)]


### Number of unique users

In [9]:
QUERY = "SELECT COUNT(DISTINCT(e.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways)e"
c.execute(QUERY)

rows = c.fetchall()

print("Number of unique users:",rows)

Number of unique users: [(3534,)]


### TOP 10 Contributors

In [10]:
QUERY = "SELECT e.user, COUNT(*) as num \
            FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e \
            GROUP BY e.user \
            ORDER BY num DESC \
            LIMIT 10"
c.execute(QUERY)

rows = c.fetchall()

print("TOP 10 Contributors:",rows)

TOP 10 Contributors: [('cirdancarpintero', 399112), ('carlosz22', 374452), ('rgbimport', 283903), ('robertogeb', 196791), ('rafaerti', 192873), ('gpesquero', 156083), ('mor', 131638), ('Iván_', 125767), ('sergionaranja', 96211), ('Cuenqui', 95691)]


### Users appearing only once (having 1 post)

In [11]:
QUERY = "SELECT COUNT(*) \
           FROM (SELECT e.user, COUNT(*) as num \
           FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e \
           GROUP BY e.user \
           HAVING num=1)  u;"
c.execute(QUERY)

rows = c.fetchall()

print("Users with only one contribution",rows)


Users with only one contribution [(984,)]


### Which are the most common type of shops in this area?

In [12]:
QUERY = "SELECT value, COUNT(*) as num \
         FROM NODES_TAGS as nt \
         WHERE nt.key = 'shop'\
         GROUP BY nt.value \
         ORDER BY num DESC \
         LIMIT 10;" 
c.execute(QUERY)

rows = c.fetchall()

print("Most typical kind of shop are :",rows)



Most typical kind of shop are : [('clothes', 1233), ('hairdresser', 1010), ('supermarket', 958), ('convenience', 856), ('bakery', 483), ('car_repair', 400), ('greengrocer', 383), ('shoes', 366), ('vacant', 306), ('optician', 283)]


We can see that clothing shops, hairdressers and supermarkets are the most typical shops that we can find in
the metropolitan area of Madrid. Now we know that clothing stores are the kind of business more represented let see
which are the brands behind these numbers.

### What are the clothing companies with more open stores in the Madrid metropolitan area?

In [13]:
QUERY = "SELECT nt1.value, COUNT(nt1.id) as count \
         FROM nodes_tags nt1 \
         JOIN (SELECT id FROM nodes_tags WHERE value = 'clothes') nt2 \
         ON nt1.id = nt2.id \
         WHERE nt1.key = 'name' \
         GROUP BY nt1.value \
         ORDER BY count DESC \
         LIMIT 20;"
        
c.execute(QUERY)

rows = c.fetchall()

print("Clothing brands with more presence in the dataset :",rows)

Clothing brands with more presence in the dataset : [('Mango', 20), ('Zara', 17), ('H&M', 13), ('Sfera', 11), ('Springfield', 11), ('Calzedonia', 10), ('Cortefiel', 8), ('Humana', 8), ('Massimo Dutti', 8), ('Amichi', 7), ('C&A', 7), ('El Ganso', 7), ('Mulaya', 7), ('Oysho', 7), ('Primark', 7), ('Bershka', 6), ('Desigual', 6), ('Stradivarius', 6), ('Pilar Prieto', 5), ('Pull & Bear', 5)]


I can recognize a lot of very well know brands... for example the ones which belongs to Inditex Group (Zara, Massimo Duti, Bershka, Stradivarius, Oyso, Pull & Bear...) which is the biggest textile company in the world and also they are a Spanish company. In my opinion, it makes a lot of sense to see a lot of open stores of this group in its origin country.

### Which are the TOP 5 Streets with more presence in the dataset?

In [14]:
QUERY = "SELECT value, COUNT(*) as num \
         FROM NODES_TAGS as nt \
         WHERE nt.key = 'street'\
         GROUP BY nt.value \
         ORDER BY num DESC \
         LIMIT 5;" 
c.execute(QUERY)

rows = c.fetchall()

print("Streets with more presence in the dataset :",rows)

Streets with more presence in the dataset : [(' Calle De Bravo Murillo', 346), (' Calle De Alcalá', 302), (' Calle De Fuencarral', 298), (' Calle De San Bernardo', 272), (' Calle Mayor', 243)]


This result have sense, as "Calle de Bravo Murillo" and "Calle de Alcalá" are the two longest streets of Madrid, furthermore, "Calle de Fuencarral" is probably the busiest street of madrid and together with "Calle de San Bernardo" and "Calle Mayor" they belong to the city center.

<a id='add'></a>

## Additional ideas about the dataset

>As it was show in the basic stastics more than 3,5k users have added information to this area, in my opinion it is a considerable number of people and after some data exploration time I noticed that not all of them are using the same criteria when they include information.

>We have some guidelines in the Wiki that should be followed by the users in order to have some consistency in the data but it seems that they are not followed by everyone.

>One of the most shocking thing that I discovered is that the info from some keys are added in different language, in this dataset: Spanish and English. I can easily prove this having a look in the postal code key of ways_tags table. I downloaded the map of postal codes for Madrid region and I can ensure that all postal code in this area have a length of 5 characters and start with "28XXX". This can be checked in the following link: http://www.madrid.org/iestadis/fijas/estructu/general/territorio/descarga/cpos.pdf

>So, I have added these conditions to the following sql query and I discovered that we have a significant number of keys with length = 5 and start with 28 whose key is "codigo_postal" which means "postcode" in Spanish.
This query also allows me to indentify that besides to the language issue, they key "postal_code" is being used, so there is more than one problem to be fixed.

In [15]:
QUERY = "SELECT key, COUNT(*) as num \
         FROM WAYS_TAGS as wt \
         WHERE (wt.value LIKE '%28%') AND (length(wt.value) = 5)\
         GROUP BY wt.key \
         ORDER BY num DESC;"

c.execute(QUERY)

rows = c.fetchall()

print("Kind of keys whose value have the following structure '28XXX' :",rows)

Kind of keys whose value have the following structure '28XXX' : [('postcode', 34471), ('codigo_postal', 911), ('postal_code', 119), ('ref', 58), ('housenumber', 3), ('city', 1), ('direction', 1), ('slope:direction', 1)]


>My proposal for avoid this situation and other related to this is select a reduced number of master users with more rights than the standard users for each area, so they will be responsible to do reviews about what other users have included. They will have the possibility to remove or accept what the standard users have propose to change. 

>The benefits for this proposal are:
><ul>
><li>Define a standarised way of working between the contributors for adding information to OpenStreetMap.
><li>Data accuracy will improve as it is understood taht the master users will use the same criteria for completing the maps.
><li>Improvements in keys categorization based in the experience of the master users which will be able to identify quicker the categorization issues of the elements.
><li>Master users could easily identify the users more likely to include wrong information based in the number of contributions who they did to the map (if it is the first contribution, for sure, it should be checked) or based in the user previous contributions.

>Nevertheless there some consideration to be taken into account in this solution. The master users should be trusted users because they will have the responsability to provide truthful information to the user which probably is one of the most important thing for OpenStreetMap. If the users cannot find reliable info, the project would have no value.

>Another relevant topic to deal is the periodicity of the reviews, the master user role should demand a considerable activity of the master users in order to attend the modification requests from other standard users. If the request are put in a never ending queue or never be attended it will no see any improvement in the data quality for the maps. 
A potential solution to easily identify how valuable is a user it could be to introduce a user rating accesible to everyone in the same way than it is done in other platforms like Amazon or forums. I think this implementation would show information about the most valuable users who highly contribution to the development of the project.

>In my opinion, these possible drawbacks would be resolved if these master rights were given to the TOP 10 contributors per area with a monthly basis review. This ranking could be elaborated measuring the number of contributions and also the rating provided by other users. The monthly basis review should be the lure for a actively contribution by the master users if they don't want to lose rights for modifying information over their favourites places.

<a id='conc'></a>
## Conclusion
From my point of view, the current data from the dataset of the metropolitan city of Madrid, Spain shows valuable information that can be analyzed and used for making some general conclusions about what things we can find in this area. But if we want to show accurate information or present exact figures this is not the best dataset.

I have found several issues in the data during the audit phase, specially related with the way about how the users include the information.I would say that the data quality of this file needs to be improved, validity and accuracy are not the best, probably this is because the high number of users which have contribute for this area (more than 3.5k) and considering that only around 28% (984 users) just contribute one time. Nevertheless, completeness of the file is quite good as we can find a lot of details about what can be found in the city.

The good thing is that there is a large room of improvement and the number of users who contributed more than one time is high, so it is only a matter of time that this map present higher quality data.
